In [1]:
%matplotlib qt

import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d
from scipy.interpolate import KroghInterpolator

In [ ]:
LABELS = ["Alt_Si3N4", "Anc_Si3N4", "lambda", "n_eff", "A_eff"]

def load_and_normalize(fname):
    with h5py.File(fname, "r") as f:
        A = f["/A"][...]
    axis5 = next((ax for ax, s in enumerate(A.shape) if s == 5), None)
    if axis5 is None:
        raise ValueError(f"No encontré eje de tamaño 5 en {fname}")
    if axis5 != A.ndim - 1:
        A = np.moveaxis(A, axis5, -1)
    return A

In [3]:
archivos = [
    "Datos_reto/Datos1.h5",
    "Datos_reto/Datos2.h5",
    "Datos_reto/Datos3.h5",
    "Datos_reto/Datos4.h5",
    "Datos_reto/Datos5.h5"
]
datasets = [load_and_normalize(fname) for fname in archivos]

def check_borders(d1, d2):
    alt1_end = np.round(d1[-1,0,0,0],6)
    alt2_start = np.round(d2[0,0,0,0],6)
    anc1_end = np.round(d1[0,-1,0,1],6)
    anc2_start = np.round(d2[0,0,0,1],6)
    print(f"Check Alt: {alt1_end} vs {alt2_start}, Check Anc: {anc1_end} vs {anc2_start}")
    if not np.isclose(alt1_end, alt2_start) and not np.isclose(anc1_end, anc2_start):
        print("⚠️ Advertencia: los bordes no coinciden perfectamente")

for i in range(len(datasets)-1):
    check_borders(datasets[i], datasets[i+1])

DatosU = np.block([
    [datasets[0]],
    [datasets[1]],
    [datasets[2]],
    [datasets[3]],
    [datasets[4]]
])

Check Alt: 0.4 vs 0.55, Check Anc: 0.875 vs 0.875
Check Alt: 0.55 vs 0.7, Check Anc: 1.25 vs 1.25
Check Alt: 0.7 vs 0.85, Check Anc: 1.625 vs 1.625
Check Alt: 0.85 vs 0.925, Check Anc: 2.0 vs 1.625
⚠️ Advertencia: los bordes no coinciden perfectamente


In [4]:
with h5py.File("Datos_reto/DatosU.h5", "w") as f_out:
    f_out.create_dataset("A", data=DatosU)

print("✅ Dataset unificado guardado en DatosU.h5, shape:", DatosU.shape)

✅ Dataset unificado guardado en DatosU.h5, shape: (50, 10, 40, 5)


In [5]:
def to_dataframe(arr):
    nA, nB, nC, _ = arr.shape
    rows = []
    for ii in range(nA):
        for jj in range(nB):
            for kk in range(nC):
                rec = {"i": ii+1, "j": jj+1, "k": kk+1}
                for p, name in enumerate(LABELS):
                    rec[name] = arr[ii, jj, kk, p]
                rows.append(rec)
    return pd.DataFrame(rows, columns=["i","j","k"]+LABELS)

df = to_dataframe(DatosU)
df["lambda_nm"] = df["lambda"] * 1000 
print(df)

        i   j   k  Alt_Si3N4  Anc_Si3N4  lambda     n_eff         A_eff  \
0       1   1   1   0.400000        0.5    0.72  1.801107  1.766958e-13   
1       1   1   2   0.416667        0.5    0.72  1.807751  1.808637e-13   
2       1   1   3   0.433333        0.5    0.72  1.813994  1.852233e-13   
3       1   1   4   0.450000        0.5    0.72  1.819495  1.894631e-13   
4       1   1   5   0.466667        0.5    0.72  1.826820  1.984774e-13   
...    ..  ..  ..        ...        ...     ...       ...           ...   
19995  50  10  36   0.925000        2.0    0.89  1.966443  1.032798e-12   
19996  50  10  37   0.943750        2.0    0.89  1.967794  1.049798e-12   
19997  50  10  38   0.962500        2.0    0.89  1.969078  1.066814e-12   
19998  50  10  39   0.981250        2.0    0.89  1.970300  1.083843e-12   
19999  50  10  40   1.000000        2.0    0.89  1.971440  1.100531e-12   

       lambda_nm  
0          720.0  
1          720.0  
2          720.0  
3          720.0  
4   

In [6]:
fig1 = plt.figure(figsize=(10,8))
ax1 = fig1.add_subplot(111, projection='3d')
p1 = ax1.scatter(df["Alt_Si3N4"], df["Anc_Si3N4"], df["lambda"],
                 c=df["n_eff"], cmap="viridis", s=10)
ax1.set_xlabel("Altura (µm)")
ax1.set_ylabel("Ancho (µm)")
ax1.set_zlabel("Lambda (µm)")
fig1.colorbar(p1, ax=ax1, label="n_eff")
ax1.set_title("n_eff")

fig2 = plt.figure(figsize=(10,8))
ax2 = fig2.add_subplot(111, projection='3d')
p2 = ax2.scatter(df["Alt_Si3N4"], df["Anc_Si3N4"], df["lambda"],
                 c=df["A_eff"], cmap="plasma", s=10)
ax2.set_xlabel("Altura (µm)")
ax2.set_ylabel("Ancho (µm)")
ax2.set_zlabel("Lambda (µm)")
fig2.colorbar(p2, ax=ax2, label="A_eff")
ax2.set_title("A_eff")

plt.show()

In [7]:
# rangos en µm
rango_p = (0.720, 0.890)   # bomba
rango_s = (1.530, 1.570)   # señal
rango_i = (0.500, 0.680)   # idler

# todas las lambdas de bombeo dentro del rango
lambdas_p = df["lambda"].unique()
lambdas_p = sorted([lp for lp in lambdas_p if rango_p[0] <= lp <= rango_p[1]])

print(f"Voy a probar con {len(lambdas_p)} Lp entre {lambdas_p[0]} y {lambdas_p[-1]} µm")

Voy a probar con 50 Lp entre 0.72 y 0.89 µm


In [8]:
resultados = []

df_filtro = df[(df["i"] == 1) & (df["j"] == 1)].sort_values("lambda")
f_interp = interp1d(df_filtro["lambda"], df_filtro["n_eff"],
                    kind="linear", fill_value="extrapolate")

# barrido
for lp in lambdas_p:
    for ls in np.linspace(rango_s[0], rango_s[1], 500):
        if (1/lp - 1/ls) <= 0:
            continue
        li = 1 / (1/lp - 1/ls)
        if rango_i[0] <= li <= rango_i[1]:
            neff_s = float(f_interp(ls))
            neff_i = float(f_interp(li))
            resultados.append({
                "lambda_p (µm)": lp,
                "lambda_s (µm)": ls,
                "lambda_i (µm)": li,
                "n_eff_s": neff_s,
                "n_eff_i": neff_i
            })

tabla = pd.DataFrame(resultados)
print("=== Tabla phase matching ===")
print(tabla)

=== Tabla phase matching ===
Empty DataFrame
Columns: []
Index: []


In [25]:
def ajustar_neff_vs_lambda(df, j_sel, k_sel, grado=3):
    df_geom = df[(df["j"]==j_sel) & (df["k"]==k_sel)].sort_values("lambda")
    if len(df_geom) <= grado:
        raise ValueError("No hay suficientes puntos para ajuste polinomial")

    x = df_geom["lambda"].values
    y = df_geom["n_eff"].values

    coef = np.polyfit(x, y, grado)
    p = np.poly1d(coef)

    x_new = np.linspace(x.min(), x.max(), 200)
    plt.figure(figsize=(8,5))
    plt.scatter(x, y, label="Datos", color="blue")
    plt.plot(x_new, p(x_new), label=f"Ajuste polinomial grado {grado}", color="red")
    plt.xlabel("Lambda (µm)")
    plt.ylabel("n_eff")
    plt.title(f"Ajuste n_eff(λ) para (j={j_sel}, k={k_sel})")
    plt.legend()
    plt.show()

    return p

In [15]:
print("Rango de λ en el dataset:", df["lambda"].min(), "–", df["lambda"].max())
print("Número de λ únicos:", len(df["lambda"].unique()))

Rango de λ en el dataset: 0.72 – 0.89
Número de λ únicos: 50


In [31]:
df_geom = df[(df["j"]==3) & (df["k"]==4)].sort_values("lambda")

'''print(df_geom[["lambda","n_eff"]])
print(len(df_geom[["lambda","n_eff"]]))'''

'print(df_geom[["lambda","n_eff"]])\nprint(len(df_geom[["lambda","n_eff"]]))'

In [41]:
j_sel, k_sel = 10, 4

df_geom = df[(df["j"]==j_sel) & (df["k"]==k_sel)]

alto = df_geom["Alt_Si3N4"].iloc[0]
ancho = df_geom["Anc_Si3N4"].iloc[0]

print(f"Geometría seleccionada: j={j_sel}, k={k_sel}")
print(f"Altura = {alto:.4f} µm")
print(f"Ancho  = {ancho:.4f} µm")

Geometría seleccionada: j=10, k=4
Altura = 0.4500 µm
Ancho  = 0.8750 µm


In [42]:
p = ajustar_neff_vs_lambda(df, j_sel, k_sel, grado=30)

def polinomio_a_str(p):
    terms = []
    for i, coef in enumerate(p.coefficients):
        exp = p.order - i
        if exp == 0:
            terms.append(f"{coef:+.4f}")
        elif exp == 1:
            terms.append(f"{coef:+.4f}·λ")
        else:
            terms.append(f"{coef:+.4f}·λ^{exp}")
    return " ".join(terms)

print("n_eff(λ) ≈", polinomio_a_str(p))

n_eff(λ) ≈ +16.9909·λ^30 -54.6476·λ^29 +36.6047·λ^28 +30.2808·λ^27 -10.6620·λ^26 -26.7933·λ^25 -13.8040·λ^24 +6.6907·λ^23 +16.5848·λ^22 +12.1406·λ^21 +0.5907·λ^20 -8.5633·λ^19 -10.1398·λ^18 -4.9972·λ^17 +2.1304·λ^16 +6.3982·λ^15 +5.7676·λ^14 +1.5743·λ^13 -2.8162·λ^12 -4.4694·λ^11 -2.6852·λ^10 +0.7626·λ^9 +3.0335·λ^8 +2.3909·λ^7 -0.3341·λ^6 -2.3728·λ^5 -1.4510·λ^4 +1.3658·λ^3 +1.7624·λ^2 -2.1382·λ +2.5935


C:\Users\david\AppData\Local\Temp\ipykernel_18544\4220940059.py:9: RankWarning: Polyfit may be poorly conditioned
  coef = np.polyfit(x, y, grado)
